## So far:
pad sentence and record ending index

But cannot index properly...

So how can I do batching??? Well, pad it with . zero. Just let it learn? How the effect of 0 will not be too big

In [1]:
from WAC import WAC
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

def getdata(name, data_dir="../data/"):
    with open(data_dir + name) as f:
        outX = []
        outy = []
        for sent in f.readlines():
            Xy = sent.split()
            outX.append(Xy[:-1])
            outy.append(int(Xy[-1]))
        #outy = torch.Tensor(outy)
    return  outX, np.array(outy)

def build_dict(trainX, testX, devX):
    import numpy as np
    train_flat = [x for item in trainX  for x in item]
    test_flat = [x for item in testX  for x in item]
    dev_flat = [x for item in devX  for x in item]

    total_unique = np.unique(train_flat + test_flat + dev_flat)
    voc_ix = {}
    for ix, word in enumerate(total_unique):
        voc_ix[word] = ix + 1

    return voc_ix

def corpus2ix(corpus, voc_ix):
    out = []
    for sent in corpus:
        out.append([voc_ix[w] for w in sent])
    return out

def pad_features(reviews_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(reviews_int), seq_length), dtype = int)
    end_ix = np.zeros(len(reviews_int), dtype = int)
    
    for i, review in enumerate(reviews_int):
        review_len = len(review)
        
        if review_len <= seq_length:
            zeroes = list(np.zeros(seq_length-review_len))
            new = review + zeroes
        elif review_len > seq_length:
            new = review[0:seq_length]
        
        features[i,:] = np.array(new)
        end_ix[i] = review_len
    
    return features, end_ix

In [2]:
start = time.time()
(trainX, trainy) = getdata('senti.train.tsv')
(testX, testy) = getdata('senti.test.tsv')
(devX, devy) = getdata('senti.dev.tsv')

## build dictionary
voc_ix = build_dict(trainX, testX, devX)

## word2ix
trainX = corpus2ix(trainX, voc_ix)
testX = corpus2ix(testX, voc_ix)
devX = corpus2ix(devX, voc_ix)

## padding
Seq_length = max([len(x) for x in trainX] + \
                 [len(x) for x in testX]+[len(x) for x in devX])
trainX,  train_endi = pad_features(trainX, Seq_length)
testX,  test_endi = pad_features(testX, Seq_length)
devX,  dev_endi = pad_features(devX, Seq_length)

train_data = TensorDataset(torch.from_numpy(trainX),\
                           torch.from_numpy(trainy),torch.from_numpy(train_endi))
test_data = TensorDataset(torch.from_numpy(testX),\
                           torch.from_numpy(testy),torch.from_numpy(test_endi))
dev_data = TensorDataset(torch.from_numpy(devX),\
                           torch.from_numpy(devy),torch.from_numpy(dev_endi))

print("runtime: {}".format(time.time() - start))

runtime: 1.4221277236938477


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import log
import torch.optim as optim
from WAC import WAC


'''
load and prepare data
'''
#(voc_ix, trainX, trainy, testX, testy, devX, devy) = data_preprocessing()
#print("finish preparing data\n")

'''
set parameters
'''
## set hyperparameters
VOCAB_SIZE = len(voc_ix) + 1
EMBEDDING_DIM = 100
n_epoch = 10
batch_size = 500
eval_per = 5000/batch_size
PATH = "../model/wac_batch.pt"

## define model
model = WAC(EMBEDDING_DIM, VOCAB_SIZE)
optimizer = optim.Adagrad(model.parameters(), lr = 1e-2, lr_decay = 1e-4)

In [11]:
## training
losses = []
accs = []
i = 0
best_dev_acc = 0

myloss = torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
start = time.time()
for epoch in range(n_epoch):
    print("epoch " + str(epoch))
    
    #dataloaders
    # make sure to SHUFFLE your data
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    for X,y,ix in train_loader:

        if i % eval_per == 0:
            print("time: {}".format(time.time() - start))
            acc = model.evaluate(dev_data.tensors)
            if acc > best_dev_acc:
                best_dev_acc = acc
                torch.save(model, PATH)
            print("accuracy on dev: " + str(acc))
            accs.append(acc)

        # Step 3. Run our forward pass.
        prob = model.forward(torch.t(X))

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        #loss_sent = - y*log(prob) - (1-y)*log(1-prob)
        loss = myloss(prob, y.float())
        #loss += loss_sent

        loss.backward()
        optimizer.step()
        model.zero_grad()
        i +=1

    losses.append(loss.item())
    runtime = time.time() - start
print("runtime: " + str(runtime) + "s")

# model_best = torch.load(PATH)
# model_best.eval()
# acc_dev = model_best.evaluate(devX, devy)
# print("best model acc on dev: " + str(acc_dev))
# acc_test = model_best.evaluate(testX, testy)
# print("best model acc on test: " + str(acc_test))

epoch 0
time: 0.013703107833862305
accuracy on dev: 0.4908256880733945


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([500])) that is different to the input size (torch.Size([500, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


time: 0.43471217155456543
accuracy on dev: 0.5091743119266054
time: 0.7742490768432617
accuracy on dev: 0.5091743119266054
time: 1.0980091094970703
accuracy on dev: 0.5114678899082569
time: 1.449465036392212
accuracy on dev: 0.5114678899082569
time: 1.8102221488952637
accuracy on dev: 0.5240825688073395
time: 2.167250156402588
accuracy on dev: 0.536697247706422
time: 2.501955032348633
accuracy on dev: 0.5412844036697247
time: 2.8400492668151855
accuracy on dev: 0.5424311926605505
time: 3.1480650901794434
accuracy on dev: 0.5607798165137615
time: 3.4833991527557373
accuracy on dev: 0.6009174311926605
time: 3.8150510787963867
accuracy on dev: 0.5848623853211009
time: 4.121809244155884
accuracy on dev: 0.6456422018348624
time: 4.525230169296265
accuracy on dev: 0.6811926605504587


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([349])) that is different to the input size (torch.Size([349, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


epoch 1
time: 4.8874900341033936
accuracy on dev: 0.676605504587156
time: 5.269345998764038
accuracy on dev: 0.6823394495412844
time: 5.60003924369812
accuracy on dev: 0.680045871559633
time: 5.925462007522583
accuracy on dev: 0.6708715596330275
time: 6.254996061325073
accuracy on dev: 0.7075688073394495
time: 6.6338889598846436
accuracy on dev: 0.7018348623853211
time: 7.044145107269287
accuracy on dev: 0.694954128440367
time: 7.417456150054932
accuracy on dev: 0.7213302752293578
time: 7.757629871368408
accuracy on dev: 0.7006880733944955
time: 8.083674192428589
accuracy on dev: 0.7064220183486238
time: 8.418361186981201
accuracy on dev: 0.7018348623853211
time: 8.726108074188232
accuracy on dev: 0.7064220183486238
time: 9.05339002609253
accuracy on dev: 0.7144495412844036
epoch 2
time: 9.360318899154663
accuracy on dev: 0.7305045871559633
time: 9.690591096878052
accuracy on dev: 0.7373853211009175
time: 10.026754140853882
accuracy on dev: 0.7350917431192661
time: 10.385966062545776
a

In [12]:
model.evaluate(test_data.tensors)

0.8050521691378364